In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
!cat ~/.comet.config

[comet]
api_key = zys8WJ1oUQDCsvZiNI9hfcEK9


In [4]:
from comet_ml import Experiment, Artifact

exp = Experiment(
    api_key='zys8WJ1oUQDCsvZiNI9hfcEK9',
    project_name='JSON_SCRAPPER_OUTPUT',
    workspace='nhl-project',
)

art = Artifact(
    name='data_2016_2020_7ecc5b717675',
    artifact_type='json_scrapper_output',
    metadata={'date': '11 nov 2023'},
)

art.add('../data/json_scrapper_output/data_2016_2020_7ecc5b717675.csv')

exp.log_artifact(art)

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nhl-project/json-scrapper-output/691a795a8da8492e9ed7bb062f4cf49e

COMET INFO: Artifact 'data_2016_2020_7ecc5b717675' version 2.0.0 created (previous was: 1.0.0)
COMET INFO: Scheduling the upload of 1 assets for a size of 180.46 MB, this can take some time
COMET INFO: Artifact 'nhl-project/data_2016_2020_7ecc5b717675:2.0.0' has started uploading asynchronously
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nhl-project/json-scrapper-output/6

In [2]:
import hydra
import os
from pathlib import Path
from omegaconf import DictConfig
import pandas as pd
from rich import print

import sys 
sys.path.append('/home/lolo/Documents/UDEM_courses/session_automne_2023/data_science/projet_NHL_repo/IFT6758_NHL/')


from utils import init_logger, verify_dotenv_file
from feature_engineering import NHL_Feature_Engineering

from ipywidgets import interact, interactive, fixed, interact_manual, widgets

verify_dotenv_file(Path('../'))
logger=init_logger('test_feature_engineering.log')

PATH_HOCKEY_COURT='../assets/hocker_court.jpg'

Loading .env file from 
/home/lolo/Documents/UDEM_courses/session_automne_2023/data_science/projet_NHL_repo/IFT6758_NHL

Loading .env file from ..

In [3]:
RAW_DATA_PATH = Path(os.getenv("DATA_FOLDER"))/ 'clean_data_2016_2020.csv'
logger.info(f"Loading raw data from {RAW_DATA_PATH}")
RAW_DATA_DF : pd.DataFrame = pd.read_csv(RAW_DATA_PATH)
data = NHL_Feature_Engineering(
    df = RAW_DATA_DF,
    distance_to_goal = True,
    angle_to_goal = True,
    is_goal = False,
    empty_net = False,
)

2023-11-07 23:53:13.599 | INFO     | __main__:<module>:2 - Loading raw data from /home/lolo/Documents/UDEM_courses/session_automne_2023/data_science/projet_NHL_repo/IFT6758_NHL/data/clean_data_2016_2020.csv


FileNotFoundError: [Errno 2] No such file or directory: '/home/lolo/Documents/UDEM_courses/session_automne_2023/data_science/projet_NHL_repo/IFT6758_NHL/data/clean_data_2016_2020.csv'

# Testing distance computing - Interactive Visual Ipywidget

In [27]:
import base64
import plotly.graph_objects as go

def plot_play_on_svg(
        play_to_plot : pd.DataFrame,
        svg_file_path,
    ):

    # Define the layout
    layout = go.Layout(
        title=f'Shots from a distance of the goal between  {play_to_plot["distance_to_goal"].min()} and {play_to_plot["distance_to_goal"].max()}',
        xaxis_title="Coordonnée X",
        yaxis_title="Coordonnée Y",
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gray', range=[-98, 98]),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gray', range=[-40, 40]),
        width=1305,
        height=715,
        title_x=0.5,
        title_y=0.9
    )

    def image_to_base64(img_path):
        with open(img_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Add the hockey rink image
    hockey_rink_img_base64 = "data:image/png;base64," + image_to_base64(PATH_HOCKEY_COURT)
    layout['images'] = [dict(
        source=hockey_rink_img_base64,
        xref="x",
        yref="y",
        x=-98,  
        y=40,  
        sizex=200,  
        sizey=80,  
        opacity=0.7,
        layer="above")
    ]

    figures_data = []

    from utils import GOAL_POSITION

    figures_data.extend([
        go.Scatter(
            x=play_to_plot['coordinateX'],
            y=play_to_plot['coordinateY'],
            mode='markers',
            marker=dict(
                size=10,
                color=play_to_plot['distance_to_goal'],
                colorscale='Viridis',
                showscale=True,
                line_width=2
            ),
            name='Shots',
            text=play_to_plot['distance_to_goal'],
            hovertemplate=
            "<b>%{text}</b><br><br>" +
            "Coordonnée X: %{x}<br>" +
            "Coordonnée Y: %{y}<br>" +
            "Distance au but: %{marker.color}<br>" +
            "<extra></extra>",
        ),
        go.Scatter(
            x=[GOAL_POSITION[0]],
            y=[GOAL_POSITION[1]],
            mode='markers',
            marker=dict(
                size=10,
                color='red',
                line_width=2
            ),
            name='Goal',
            text='Goal',
            hovertemplate=
            "<b>%{text}</b><br><br>" +
            "Coordonnée X: %{x}<br>" +
            "Coordonnée Y: %{y}<br>" +
            "<extra></extra>",
        )
    ])

    fig = go.Figure(data=figures_data, layout=layout)
    
    return fig

In [28]:
def display_on_hockey_court_play_dist_range(data_plays, range_distance, svg_file_path : Path):

    df_to_plot = data_plays[(data_plays['distance_to_goal'] > range_distance[0]) & (data_plays['distance_to_goal'] < range_distance[1])]
    fig = plot_play_on_svg(df_to_plot, svg_file_path)
    fig.show()
    print(df_to_plot.shape, df_to_plot.describe())

In [29]:
data.df['season']

0         2016
1         2016
2         2016
3         2016
4         2016
          ... 
387824    2020
387825    2020
387826    2020
387827    2020
387828    2020
Name: season, Length: 387829, dtype: int64

In [30]:
from copy import deepcopy


dist_df = deepcopy(data.df[data.df['season']==2016][['coordinateX', 'coordinateY', 'distance_to_goal']])

MEAN_dist_df = dist_df['distance_to_goal'].mean()
STD_dist_df = dist_df['distance_to_goal'].std()

interact(
    display_on_hockey_court_play_dist_range,
    data_plays = fixed(dist_df),
    range_distance = widgets.FloatRangeSlider(
        value=[MEAN_dist_df-0.1, MEAN_dist_df+0.1],
        min=dist_df['distance_to_goal'].min(),
        max=dist_df['distance_to_goal'].max(),
        step=0.1,
        description='Choose a range of distance :',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    ),
    svg_file_path = fixed(PATH_HOCKEY_COURT)
)

interactive(children=(FloatRangeSlider(value=(43.03391127503297, 43.23391127503297), continuous_update=False, …

<function __main__.display_on_hockey_court_play_dist_range(data_plays, range_distance, svg_file_path: pathlib.Path)>

# Testing distance computing - Interactive Visual Ipywidget

In [20]:
import base64
import plotly.graph_objects as go

def plot_play_on_svg(
        play_to_plot : pd.DataFrame,
        svg_file_path,
    ):

    # Define the layout
    layout = go.Layout(
        title=f'Shots with an angle from the goal between  {play_to_plot["angle_to_goal"].min()} and {play_to_plot["angle_to_goal"].max()}',
        xaxis_title="Coordonnée X",
        yaxis_title="Coordonnée Y",
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gray', range=[-98, 98]),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gray', range=[-40, 40]),
        width=1305,
        height=715,
        title_x=0.5,
        title_y=0.9
    )

    def image_to_base64(img_path):
        with open(img_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # Add the hockey rink image
    hockey_rink_img_base64 = "data:image/png;base64," + image_to_base64(PATH_HOCKEY_COURT)
    layout['images'] = [dict(
        source=hockey_rink_img_base64,
        xref="x",
        yref="y",
        x=-98,  
        y=40,  
        sizex=200,  
        sizey=80,  
        opacity=0.7,
        layer="above")
    ]

    figures_data = []

    from utils import GOAL_POSITION

    figures_data.extend([
        go.Scatter(
            x=play_to_plot['coordinateX'],
            y=play_to_plot['coordinateY'],
            mode='markers',
            marker=dict(
                size=10,
                color=play_to_plot['angle_to_goal'],
                colorscale='Viridis',
                showscale=True,
                line_width=2
            ),
            name='Shots',
            text=play_to_plot['angle_to_goal'],
            hovertemplate=
            "<b>%{text}</b><br><br>" +
            "Coordonnée X: %{x}<br>" +
            "Coordonnée Y: %{y}<br>" +
            "Angle au but: %{marker.color}<br>" +
            "<extra></extra>",
        ),
        go.Scatter(
            x=[GOAL_POSITION[0]],
            y=[GOAL_POSITION[1]],
            mode='markers',
            marker=dict(
                size=10,
                color='red',
                line_width=2
            ),
            name='Goal',
            text='Goal',
            hovertemplate=
            "<b>%{text}</b><br><br>" +
            "Coordonnée X: %{x}<br>" +
            "Coordonnée Y: %{y}<br>" +
            "<extra></extra>",
        )
    ])

    fig = go.Figure(data=figures_data, layout=layout)
    
    return fig

In [21]:
def display_on_hockey_court_play(data_plays, range_angles, svg_file_path : Path):

    df_to_plot = data_plays[(data_plays['angle_to_goal'] > range_angles[0]) & (data_plays['angle_to_goal'] < range_angles[1])]
    fig = plot_play_on_svg(df_to_plot, svg_file_path)
    fig.show()
    print(df_to_plot.shape, df_to_plot.describe())

In [22]:
from copy import deepcopy


dist_df = deepcopy(data.df[data.df['season']==2016][['coordinateX', 'coordinateY', 'angle_to_goal']])
MEAN_dist_df = dist_df['angle_to_goal'].mean()
STD_dist_df = dist_df['angle_to_goal'].std()

interact(
    display_on_hockey_court_play,
    data_plays = fixed(dist_df),
    range_angles= widgets.FloatRangeSlider(
        value=[MEAN_dist_df-0.1, MEAN_dist_df+0.1],
        min=dist_df['angle_to_goal'].min(),
        max=dist_df['angle_to_goal'].max(),
        step=0.1,
        description='Choose a range of distance :',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    ),
    svg_file_path = fixed(PATH_HOCKEY_COURT)
)

interactive(children=(FloatRangeSlider(value=(0.38293183086703486, 0.5829318308670349), continuous_update=Fals…

<function __main__.display_on_hockey_court_play(data_plays, range_angles, svg_file_path: pathlib.Path)>